In [1]:
import os
import csv
import requests


csv_path = "image_links.csv"
output_dir = "data"

with open(csv_path, newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        folder = row["folder"]
        filename = row["filename"]
        github_link = row["github_link"]
        save_folder = os.path.join(output_dir, folder)
        os.makedirs(save_folder, exist_ok=True)
        save_path = os.path.join(save_folder, filename)
        if not os.path.exists(save_path):
            try:
                resp = requests.get(github_link)
                if resp.status_code == 200:
                    with open(save_path, "wb") as imgf:
                        imgf.write(resp.content)
                else:
                    print(f"Failed to download {github_link} (status {resp.status_code})")
            except Exception as e:
                print(f"Error downloading {github_link}: {e}")
        else:
            print(f"Already exists: {save_path}")

Already exists: data/brain_tumor/brain_tumor_1710.jpg
Already exists: data/brain_tumor/brain_tumor_1501.jpg
Already exists: data/brain_tumor/brain_tumor_1823.jpg
Already exists: data/brain_tumor/brain_tumor_0249.jpg
Already exists: data/brain_tumor/brain_tumor_0279.jpg
Already exists: data/brain_tumor/brain_tumor_0156.jpg
Already exists: data/brain_tumor/brain_tumor_0987.jpg
Already exists: data/brain_tumor/brain_tumor_1257.jpg
Already exists: data/brain_tumor/brain_tumor_0218.jpg
Already exists: data/brain_tumor/brain_tumor_1183.jpg
Already exists: data/brain_tumor/brain_tumor_0945.jpg
Already exists: data/brain_tumor/brain_tumor_1589.jpg
Already exists: data/brain_tumor/brain_tumor_0766.jpg
Already exists: data/brain_tumor/brain_tumor_0017.jpg
Already exists: data/brain_tumor/brain_tumor_0491.jpg
Already exists: data/brain_tumor/brain_tumor_0953.jpg
Already exists: data/brain_tumor/brain_tumor_1391.jpg
Already exists: data/brain_tumor/brain_tumor_0617.jpg
Already exists: data/brain_t

In [2]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
NUM_EPOCHS = 3
IMAGE_SIZE = 512

In [3]:
import tensorflow as tf

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            directory='data',
            labels='inferred',
            label_mode='int',
            image_size=(IMAGE_SIZE, IMAGE_SIZE),
            batch_size=BATCH_SIZE,
            shuffle=True,
            seed=42,
            color_mode='rgb', #Although the images are grayscale, we need to set the color_mode to 'rgb' for the model to work properly.
            pad_to_aspect_ratio=True,
            validation_split=0.2,
            subset='training',
        )

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            directory='data',
            labels='inferred',
            label_mode='int',
            image_size=(IMAGE_SIZE, IMAGE_SIZE),
            batch_size=BATCH_SIZE,
            shuffle=True,
            seed=42,
            color_mode='rgb',
            pad_to_aspect_ratio=True,
            validation_split=0.2,
            subset='validation',
        )                                                 

Found 1514 files belonging to 3 classes.
Using 1212 files for training.
Found 1514 files belonging to 3 classes.
Using 302 files for validation.


2026-02-23 06:28:31.649871: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-02-23 06:28:31.649895: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2026-02-23 06:28:31.649899: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
I0000 00:00:1771856911.649912 17883003 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1771856911.649937 17883003 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Fine-Tuning ResNet

In [4]:
base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
        include_top=False, #Final prediction layer will be replaced with custom one
    )

base_model.trainable = False #We will only train the custom layer we add on top of the base model

resnet = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Lambda(tf.keras.applications.resnet.preprocess_input),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax', name='output')
])

In [5]:
resnet.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 16, 16, 2048)   │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,593,859 (90.00 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [7]:
resnet.fit(train_dataset, epochs=NUM_EPOCHS)

Epoch 1/3


2026-02-23 06:28:33.197259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


38/38 ━━━━━━━━━━━━━━━━━━━━ 21s 510ms/step - accuracy: 0.2553 - loss: 1.3252
Epoch 2/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 19s 482ms/step - accuracy: 0.4378 - loss: 1.0784
Epoch 3/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 20s 488ms/step - accuracy: 0.5579 - loss: 0.9275


### Fine-Tuning EfficientNet

In [8]:
base_model = tf.keras.applications.EfficientNetB5(
        weights='imagenet',
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
        include_top=False, #Final prediction layer will be replaced with custom one
    )

base_model.trainable = False #We will only train the custom layer we add on top of the base model

efficientnet = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    #EfficientNet has the preprocessing function built in
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax', name='output')
])

In [9]:
efficientnet.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb5 (Functional)     │ (None, 16, 16, 2048)   │    28,513,527 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,519,674 (108.79 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 28,513,527 (108.77 MB)

In [10]:
efficientnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [11]:
efficientnet.fit(train_dataset, epochs=NUM_EPOCHS)

Epoch 1/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.3740 - loss: 1.1109
Epoch 2/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.5764 - loss: 0.9772
Epoch 3/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.6950 - loss: 0.8853


These are just two of the many existing architectures you can use through TensorFlow. Most of them have similar syntax to create the model and then fine-tune it.

### Comparing the Two

In [12]:
print(resnet.evaluate(test_dataset, return_dict=True))
print(efficientnet.evaluate(test_dataset, return_dict=True))

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 535ms/step - accuracy: 0.6762 - loss: 0.7838
{'accuracy': 0.7152317762374878, 'loss': 0.7615852952003479}
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.8073 - loss: 0.8064
{'accuracy': 0.807947039604187, 'loss': 0.7939189672470093}


### Save the Models

Note: Since the model files are quite large (~110 MB), they have not been pushed to Github. For model files like this, Git Large File Storage is usually used.

In [13]:
import os

try:
    os.mkdir('models')
except FileExistsError:
    pass

resnet.save('models/finetuned_resnet.keras')
efficientnet.save('models/finetuned_efficientnet.keras')